In [ ]:
!pip install google-cloud-speech google-cloud

In [54]:
BUCKET_NAME = "mnlee-stt" # @param {type:"string"}

In [47]:
wav_ko_kr = f"gs://{BUCKET_NAME}/robot/ko-KR.wav"
wav_en_us = f"gs://{BUCKET_NAME}/robot/en-US.wav"
wav_de_de = f"gs://{BUCKET_NAME}/robot/de-DE.wav"
wav_es_es = f"gs://{BUCKET_NAME}/robot/es-ES.wav"
wav_fr_fr = f"gs://{BUCKET_NAME}/robot/fr-FR.wav"
wav_ja_jp = f"gs://{BUCKET_NAME}/robot/ja-JP.wav"
wav_ms_my = f"gs://{BUCKET_NAME}/robot/ms-MY.wav"
wav_vi_vn = f"gs://{BUCKET_NAME}/robot/vi-VN.wav"
wav_sv_se = f"gs://{BUCKET_NAME}/robot/sv-SE.wav"

In [38]:
from google.cloud import speech_v1p1beta1 as speech
#from google.cloud import speech_v2 as speech

#def stt(wav_uri) -> speech.RecognizeResponse:
def stt(wav_uri):
    # Instantiates a client
    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=wav_uri)

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code="en-US",
        #sample_rate_hertz=44100,
        alternative_language_codes=["ko-KR","de-DE","es-ES","fr-FR","ja-JP","ms-MY","vi-VN","sv-SE","zh-CN"]
    )

    # Detects speech in the audio file
    response = client.recognize(config=config, audio=audio)
    #print(response)
    for result in response.results:
        print("Transcript: {}".format(result.alternatives[0].transcript))
        print("lang code: {}".format(result.language_code))
          
    #return response

In [39]:
stt(wav_ko_kr)

Transcript: 안녕하세요 다국어 지원 테스트입니다
lang code: ko-kr


In [56]:
stt(wav_en_us)

Transcript: hello this is a multilingual support test
lang code: en-us


In [44]:
stt(wav_de_de)

Transcript: hallo dies ist ein mehrsprachiger Support Test
lang code: de-de


In [45]:
stt(wav_es_es)

Transcript: hola esta es una prueba de soporte multilingüe
lang code: es-es


In [48]:
stt(wav_fr_fr)

Transcript: bonjour Il s'agit d'un test de support multilingue
lang code: fr-fr


In [49]:
stt(wav_ja_jp)

Transcript: こんにちは多言語サポートテストです
lang code: ja-jp


In [50]:
stt(wav_ms_my)

Transcript: Helo ini ialah ujian sokongan berbilang bahasa
lang code: ms-my


In [52]:
stt(wav_vi_vn)

Transcript: Sing Tao Daily by Kim Chattahoochee
lang code: en-us


In [53]:
stt(wav_sv_se)

Transcript: Helvete Facebook IT support Test
lang code: de-de
